In [92]:
import numpy as np
import scipy
from scipy.stats import norm
import math
import random
import csv
import pandas as pd

In [100]:
class mykMeans:

    def __init__(self,k,n,d):
        # k is the number of clusters to form
        self.k = k
        # centroid points
        self.centroid = np.zeros(k*d).reshape(k,d)
        # weights are the how much each feature matters
        self.weights = np.ones(d)
        # d is the dimension of the feature vector
        self.d = d
        # n is the number of the samples we have
        self.n = n
        #cluster is the stored value of the data
        self.cluster = np.zeros(n)

    def showValues(self):
        print("k:",self.k)
        print("n:",self.n)
        print("d:",self.d)

    def weights(self,data):
        for i in range (0,self.d):
            # for now the weights are set as 1
            self.weights[i]=1

    def dis(self,data,centroidIndex):
        distance =0
        #using euclidean distance with weights in this case
        for i in range (0,self.d):
            if (i==1):
                distance = distance + float(((float(data[i])-self.centroid[centroidIndex][i])**2))
            else:
                if (int(data[i])==self.centroid[centroidIndex][i]):
                    distance = distance + 1
        return distance
    
    def updateClass(self,data):
#         print('n in update class: ',self.n)
        for i in range (0,self.n):
            minDistance = math.inf
#             print("i in updateClass: ",i)
            for j in range (0,self.k):
                tempDist = self.dis(data[i],j)
                if(tempDist<minDistance):
                    self.cluster[i] = j
                    minDistance = tempDist
                    # print('cluster: ',j)
        # self.printAllClasses()

    def updateCentroids(self,data):
        converged = True
        tempCentroid = np.zeros(self.k*self.d).reshape(self.k,self.d)
        tempCnumbers = np.zeros(self.k)
        for i in range(0,self.n):
            clusIndex = (int)(self.cluster[i])
            # print('clusindex for i:',i,clusIndex)
            tempCnumbers[clusIndex]+=1
            for t in range(len(tempCentroid[clusIndex])):
                tempCentroid[clusIndex][t]+= float(data[i][t])
        for i in range(0,self.k):
            t = (tempCentroid[i]/tempCnumbers[i])==self.centroid[i]
            for j in range(0,self.d):
                if(t[j]==False):
                    converged = False
                    break
                self.centroid[i] = tempCentroid[i]/tempCnumbers[i]
        return converged

    def initializeCentroids(self,data):
        #right now it takes only the first k elements
        for i in range(0,self.k):
            self.centroid[i] = data[random.randint(0,self.n-1)]

    def initKMeansPlusPlus(self,data):
        ##KMeans++ implementation
        self.centroid[0] = data[random.randint(0,self.n-1)]
        for i in range(1,self.k):
            distArr = []
            for j in range(self.n):
                mindist = math.inf
                for k in range(i):
                    tempDist = self.dis(data[j],k)
                    if mindist>tempDist:
                        mindist=tempDist
                distArr.append(mindist)
            distArr = np.array(distArr)
            probs = distArr/(np.sum(distArr))
            cumprobs = np.cumsum(probs)
            r = scipy.rand()
            for s,t in enumerate(cumprobs):
                if r<t:
                    self.centroid[i] = data[s]
                    break
            
                
    def classify(self,data,max_iter,kmplusplus=0):
        convergence = False
        if(kmplusplus==0):
            self.initializeCentroids(data)
        else:
            print("KMEANS++")
            self.initKMeansPlusPlus(data)
        p=0
        while((convergence==False) and (p!=max_iter)):
            #print(p)
            p+=1
            self.updateClass(data)
            convergence = self.updateCentroids(data)
        print("Classification Done")
#         print("Now printing cluster values:")
#         self.printAllClasses()

    def getClass(self,data):
        minDistance = math.inf
        clusterNumber = -1
        for j in range (0,self.k):
            tempDist = self.dis(data,j)
            if(tempDist<minDistance):
                clusterNumber = j
                minDistance = tempDist
        return clusterNumber
    
    def getClassForAll(self,data):
        n = data.shape[0]
        ansList=[]
        for i in range(n):
            ans = self.getClass(data[i])
            ansList.append(ans)
#             print(ans)
        return ansList
        
    def printAllClasses(self):
        for i in range(0,self.n):
            print(self.cluster[i])

In [99]:
with open('../railwayBookingList.csv','r') as csv_file:
    csv_reader = list(csv.reader(csv_file,delimiter=","))
#     csv_dicReader = csv.DictReader(csv_file)
    my_data = np.array(csv_reader)
    
    test=[]
    
    for i in range(1,len(my_data)):
        #print (my_data[i])
        a = my_data[i][1]
        my_data[i][1]= my_data[i][6]
        my_data[i][6]= a
        #exchange age category with class
        
        if (my_data[i][5]=='male'):
            my_data[i][5]=1
        else:
            my_data[i][5]=0
        
        if (my_data[i][4]=='FIRST_AC'):
            my_data[i][4]=1
        elif (my_data[i][4]=='NO_PREF'):
            my_data[i][4]=0
        elif (my_data[i][4]=='SECOND_AC'):
            my_data[i][4]=2
        else:
            my_data[i][4]=3
        test.append(np.delete(my_data[i],[0,6]))


otest = np.asarray(test)
total = [0,0,0,0,0]
for i in otest:
    for j in range(5):
        total[j] = total[j] + float(i[j])

print (total)
mean = []
for k in range(len(total)):
    mean.append(float(total[k]/1310.0))

print(mean)
    
data = [0,0,0,0,0]
for i in otest:
    for j in range(5):
        data[j] = data[j] + (float(i[j])-mean[j])**2.0
print(data)

variance= []
for k in range(len(total)):
    variance.append(float(data[k]/1310.0))

print (variance)   


for i in range(len(otest)):
    #for j in range(5):
        otest[i][1] = float((float(otest[i][1])-mean[1])/variance[1])
print(my_data)
print (otest)

[2666.0, 1187333.0, 1157.0, 3004.0, 844.0]
[2.0351145038167937, 906.36106870229, 0.883206106870229, 2.2931297709923664, 0.6442748091603053]
[3650.384732824454, 181390576.21449652, 3281.130534351174, 923.4381679389444, 300.23206106870134]
[2.7865532311637056, 138466.08871335612, 2.504679797214637, 0.7049146320144614, 0.22918477944175675]
[['caseID' 'boarded' 'budget' ... 'preferredClass' 'sex' 'age']
 ['111131089' '2' '2201' ... '1' '0' '0']
 ['2489059216' '0' '1775' ... '1' '1' '0']
 ...
 ['956944496' '2' '710' ... '3' '1' '1']
 ['3100624207' '2' '710' ... '3' '1' '1']
 ['3068255754' '0' '710' ... '0' '1' '1']]
[['2' '0.009349862795' '0' '1' '0']
 ['0' '0.006273297233' '3' '1' '1']
 ['0' '0.006273297233' '3' '1' '0']
 ...
 ['2' '-0.00141811667' '0' '3' '1']
 ['2' '-0.00141811667' '0' '3' '1']
 ['0' '-0.00141811667' '0' '0' '1']]


In [101]:
obj = mykMeans(2,1310,5)
for mn in range(100,1000,50):
    obj.classify(otest,mn,0)
    ansList = obj.getClassForAll(otest)
    #print (ansList)
    err = 0
    for i in range(len(otest)):
        if ansList[i]!=int(my_data[i+1][6]):
            err = err + 1
    print (err)

Classification Done
953
Classification Done
779
Classification Done
373
Classification Done
962
Classification Done
954
Classification Done
829
Classification Done
558
Classification Done
365
Classification Done
571
Classification Done
947
Classification Done
536
Classification Done
926
Classification Done
764
Classification Done
736
Classification Done
426
Classification Done
586
Classification Done
346
Classification Done
864


In [103]:
obj = mykMeans(2,1310,5)
for mn in range(100,1000,50):
    obj.classify(otest,mn,1)
    ansList = obj.getClassForAll(otest)
    #print (ansList)
    err = 0
    for i in range(len(otest)):
        if ansList[i]!=int(my_data[i+1][6]):
            err = err + 1
    print (err)

KMEANS++
Classification Done
945
KMEANS++
Classification Done
838
KMEANS++
Classification Done
294
KMEANS++
Classification Done
797
KMEANS++
Classification Done
720
KMEANS++
Classification Done
805
KMEANS++
Classification Done
836
KMEANS++
Classification Done
662
KMEANS++
Classification Done
973
KMEANS++
Classification Done
863
KMEANS++
Classification Done
774
KMEANS++
Classification Done
818
KMEANS++
Classification Done
797
KMEANS++
Classification Done
527
KMEANS++
Classification Done
419
KMEANS++
Classification Done
427
KMEANS++
Classification Done
741
KMEANS++


E:\anaconda\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in true_divide


Classification Done
810


In [468]:
### below is the scikit one===================================================================================================

In [102]:
from sklearn.cluster import KMeans
skmeans =KMeans(n_clusters=2)
skmeans.fit(otest)
ans = skmeans.labels_
errcount=0
for p in range(len(ans)):
    if ans[p]!=int(my_data[p+1][6]):
        errcount+=1
print(errcount)

724
